In [1]:
# import libraries
import ppi_py as ppi
import numpy as np
import statsmodels as sm
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# define the function to simulate data
def simulate_ols_data(beta, ppi_corr, bias, n, N):
    """
    Simulate data for the OLS example

    Args:
        beta (ndarray): regression coefficients, shape d + 1
        ppi_corr (float): PPI correlation
        bias (float): bias to appl to all coefficients
        n (int): number of labeled samples
        N (int): number of unlabeled samples

    Returns:
        X (ndarray): covariates for labeled data , shape n x (d + 1)
        Y (ndarray): labels, shape n
        Yhat (ndarray): predictions on labeled data, shape n
        X_unlabeled (ndarray): covariates for unlabeled data, shape N x (d + 1)
        Yhat_unlabeled (ndarray): predictions on unlabeled data set shape N
    """

    d = len(beta) - 1

    X = np.random.normal(size=(n, d+1))/(d+1)**0.5
    X[:, 0] = 1

    X_unlabeled = np.random.normal(size=(N, d+1))/(d+1)**0.5
    X_unlabeled[:, 0] = 1

    max_ppi_corr = 1/(2*bias**2+1)**0.5
    max_bias = (0.5*((1/ppi_corr)**2 - 1))**0.5
    assert ppi_corr <= max_ppi_corr, f"ppi_corr must be less than {max_ppi_corr} or bias must be less than {max_bias}"

    rho = ppi_corr * (2*bias**2+1)**0.5

    Z = np.random.normal(size=n)
    Y = X @ beta + rho**0.5 * Z + (1 - rho) ** 0.5 * np.random.normal(size=n)
    Yhat = X @ (beta + bias) + rho**0.5 * Z + (1 - rho) ** 0.5 * np.random.normal(size=n)
    Yhat_unlabeled = X_unlabeled @ (beta + bias) + np.random.normal(size=N)

    return X, Y, Yhat, X_unlabeled, Yhat_unlabeled



In [3]:
# true intercept and slope
param = 1.
beta = np.array([param, param])

# bias 
bias = 0.1

# PPI correlation
ppi_corr = 0.9

# sample size human subjects
ns = [1000]

# multiples of human subjects sample size
ks = list([0.1, 0.25, 0.5, 0.75]) + list(np.arange(1, 10.5, 0.5))

# sample size silicon subjects 
Ns = [int(n * k) for n in ns for k in ks]

# number of repetitions for combinations of n and N
reps = 1000

### Simulation over N

In this simulation, we examine how bias, standard error, coverage, and RMSE vary as a function of the number of silicon subjects $N$. We hold the number of human subjects and the PPI correlation fixed at $n=1000$ and $\tilde \rho = 0.5$, respectively. 

In [4]:
# initialize list to collect results
results = []

# loop over repetitions
for r in range(reps):

  # keep track of progress
  if (r+1) % 25 == 0 or r == 0:
    print(f"Repetition {r+1} out of {reps}")

  # loop over the combinations of n and N
  for n in ns:      

    for N in Ns:
      # simulate data
      X, Y, Yhat, X_unlabeled, Yhat_unlabeled = simulate_ols_data(beta, ppi_corr, bias, n, N)

      # get silicon sampling estimates
      mod_sil = sm.regression.linear_model.OLS(Yhat_unlabeled, X_unlabeled).fit()
      beta_sil = mod_sil.params[1]
      se_sil = mod_sil.bse[1]

      beta_ci_sil = mod_sil.conf_int(alpha=0.05)[1]
      lower_sil = beta_ci_sil[0]
      upper_sil = beta_ci_sil[1]

      # get coefficient estimate from mixed-subjects PPI
      mod_ppi = ppi.ppi_ols_pointestimate(
        X, 
        Y, 
        Yhat, 
        X_unlabeled, 
        Yhat_unlabeled)
      beta_ppi = mod_ppi[1]

      # compute PPI standard error from CIs
      beta_ci_ppi = ppi.ppi_ols_ci(X=X, Y=Y, Yhat=Yhat, X_unlabeled=X_unlabeled, Yhat_unlabeled=Yhat_unlabeled, alpha=0.05)
      lower_ppi = beta_ci_ppi[0][1]
      upper_ppi = beta_ci_ppi[1][1]
      se_ppi = (upper_ppi - lower_ppi) / (2 * 1.96)

      # report whether the true value is within the confidence interval
      ci_ppi_covers_param = int(lower_ppi <= param <= upper_ppi)
      ci_sil_covers_param = int(lower_sil <= param <= upper_sil)

      # append the results to the list as a dictionary
      results.append({
          'beta_ppi': beta_ppi,
          'se_ppi': se_ppi,
          'lower_ppi': lower_ppi,
          'upper_ppi': upper_ppi,
          'coverage_ppi': ci_ppi_covers_param,
          'beta_sil': beta_sil,
          'se_sil': se_sil,
          'lower_sil': lower_sil,
          'upper_sil': upper_sil,
          'coverage_sil': ci_sil_covers_param,
          'n': n,
          'N': N,
          'param': param,
          'bias': bias,
          'ppi_corr': ppi_corr,
          'rep': r+1
      })
  
# convert list of dictionaries to a pandas DataFrame
result_df = pd.DataFrame(results).sort_values(by=['rep','n','N','ppi_corr'], ascending=True)
result_df['repetitions'] = reps

# Group by 'n' and 'N', then calculate mean across repetitions
stats = ['beta_ppi','se_ppi','lower_ppi','upper_ppi','coverage_ppi',
         'beta_sil','se_sil','lower_sil','upper_sil','coverage_sil']
df = result_df.groupby(['n','N','ppi_corr','repetitions'])[stats].mean().reset_index()

# Calculate bias columns
df['bias_ppi'] = df['beta_ppi'] - 1
df['bias_sil'] = df['beta_sil'] - 1
df['rmse_ppi'] = np.sqrt(df['bias_ppi']**2 + df['se_ppi']**2)
df['rmse_sil'] = np.sqrt(df['bias_sil']**2 + df['se_sil']**2)

# Save averaged simulation results to compressed csv file
df.to_csv("../Data/simulation_study_N.csv.gz", compression="gzip", index=False)

Repetition 1 out of 1000
Repetition 25 out of 1000
Repetition 50 out of 1000
Repetition 75 out of 1000
Repetition 100 out of 1000
Repetition 125 out of 1000
Repetition 150 out of 1000
Repetition 175 out of 1000
Repetition 200 out of 1000
Repetition 225 out of 1000
Repetition 250 out of 1000
Repetition 275 out of 1000
Repetition 300 out of 1000
Repetition 325 out of 1000
Repetition 350 out of 1000
Repetition 375 out of 1000
Repetition 400 out of 1000
Repetition 425 out of 1000
Repetition 450 out of 1000
Repetition 475 out of 1000
Repetition 500 out of 1000
Repetition 525 out of 1000
Repetition 550 out of 1000
Repetition 575 out of 1000
Repetition 600 out of 1000
Repetition 625 out of 1000
Repetition 650 out of 1000
Repetition 675 out of 1000
Repetition 700 out of 1000
Repetition 725 out of 1000
Repetition 750 out of 1000
Repetition 775 out of 1000
Repetition 800 out of 1000
Repetition 825 out of 1000
Repetition 850 out of 1000
Repetition 875 out of 1000
Repetition 900 out of 1000
Repeti